# Predict Sales
This notebook does a grid search on the 3 gradient boosting models for both the 1st level features.

In [53]:
import numpy as np
import pandas as pd
from pandas.plotting import scatter_matrix
import gc  # For garbage collection to free up memory
import tqdm
from tqdm import tqdm_notebook
from datetime import datetime
from itertools import product  # Cartesian product, equivalent to a nested for-loop
import matplotlib.pyplot as plt

import sklearn
import lightgbm as lgb
import xgboost as xgb
import catboost as cb
import matplotlib.pyplot as plt

from xgboost.sklearn import XGBRegressor  # Scikit-Learn wrapper around XGBoost
from xgboost import plot_importance  # Will enable us to see the feature importance
from catboost import CatBoostRegressor

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from math import sqrt

for p in [np, pd, sklearn, lgb, xgb, cb, tqdm]:
    print (p.__name__, p.__version__)
    
# Define function to turn all 64-bit variables in a dataframe into 32-bit variables to save memory
def downcast_dtypes(df):
    '''
        Changes column types in the dataframe: 
                
                `float64` type to `float32`
                `int64`   type to `int32`
    '''
    
    # Select columns to downcast
    float_cols = [c for c in df if df[c].dtype == "float64"]
    int_cols =   [c for c in df if df[c].dtype == "int64"]
    
    # Downcast
    df[float_cols] = df[float_cols].astype(np.float32)
    df[int_cols]   = df[int_cols].astype(np.int32)
    
    return df

numpy 1.13.3
pandas 0.20.3+0.g3a7f956.dirty
sklearn 0.19.0
lightgbm 2.1.0
xgboost 0.7
catboost 0.6.1.1
tqdm 4.19.4


# Read in the Data

In [54]:
# Read in the sales data
sales = pd.read_csv('./sales.csv')

print(sales.tail())

         shop_id  item_id  date_block_num  item_cnt_day  item_price  \
6639289       45    18454              34           0.0         0.0   
6639290       45    16188              34           0.0         0.0   
6639291       45    15757              34           0.0         0.0   
6639292       45    19648              34           0.0         0.0   
6639293       45      969              34           0.0         0.0   

         item_category_id  item_id_mean_item_price  item_id_sum_item_cnt_day  \
6639289                55                      0.0                       0.0   
6639290                64                      0.0                       0.0   
6639291                55                      0.0                       0.0   
6639292                40                      0.0                       0.0   
6639293                37                      0.0                       0.0   

         item_id_mean_item_cnt_day  shop_id_mean_item_price  \
6639289                      

# Prep for training

In [55]:
# Save `date_block_num`, as we can't use them as features, but will need them to split the dataset into parts 
dates = sales['date_block_num']

# Note that the sales dataset has lag variables going to block 34 (which would be the actual submission block for
# the contest) - but the item_cnt_day is zeroed out since that is what we predict.  Therefore we will only use out
# to block 33 (which we will use as our test block)
last_block = dates.max() - 1
print('Test `date_block_num` is %d' % last_block)

Test `date_block_num` is 33


In [56]:
dates_train = dates[dates <  (last_block)]
dates_test  = dates[dates == (last_block)]

X_train = sales.loc[dates <  (last_block)].drop(cols_to_drop, axis=1)
X_test =  sales.loc[dates == (last_block)].drop(cols_to_drop, axis=1)
print(X_train.shape)
print(X_test.shape)

y_train = sales.loc[dates <  (last_block), 'item_cnt_day'].values
y_test =  sales.loc[dates == (last_block), 'item_cnt_day'].values  # y_test is what we need to predict for the contest
print("y_train head: ", y_train)
print("y_test head: ", y_test)

(6186922, 64)
(238172, 64)
y_train head:  [  4.   3.  14. ...,   0.   0.   0.]
y_test head:  [ 1.  1.  2. ...,  0.  0.  0.]


# First level models
We will use scheme f from the class since we have a time component.  We use first level models to build two datasets: test meta-features and 2nd level train-metafeatures.

## Test meta-features
First run grid search on the numeric columns and get predictions for the validation month

In [61]:
# Do a grid search on LightGBM
param_grid = {
    'learning_rate': [0.025, 0.05, 0.1],
    'n_estimators': [70, 75, 80],  # Number of boosted trees to fit
}

lgbmodel = lgb.LGBMRegressor(device="gpu", gpu_device_id=0, n_jobs=16, max_bin=63, num_leaves=2**7)

gbm = GridSearchCV(lgbmodel, param_grid)

start = datetime.now()

gbm.fit(X_train, y_train)

stop = datetime.now()

print("Ellapsed time = ", stop - start)
print("Best parameters found by grid search are: ", gbm.best_params_)
print("Best score found by grid search is: ", gbm.best_score_)

Ellapsed time =  0:50:15.930650
Best parameters found by grid search are:  {'learning_rate': 0.05, 'n_estimators': 75}
Best score found by grid search is:  0.448940839524


In [64]:
# Next a grid search on XGBoost
param_grid = {
    'max_depth': [8, 9, 10],
    'learning_rate': [0.025, 0.05, 0.1],
    'n_estimators': [75, 85, 95]
}

xgbmodel = XGBRegressor(gamma=0, # Minimum loss reduction required to make a further partition on a leaf node of the tree
               min_child_weight=1, # Minimum sum of instance weight(hessian) needed in a child
               subsample=0.8, # Subsample ratio of the training instance
               colsample_bytree=0.8, # Subsample ratio of columns when constructing each tree 
               colsample_bylevel=0.8, # Subsample ratio of columns for each split, in each level
               reg_alpha=0, # L1 regularization weights
               reg_lambda=1, # L2 regularization weights
               scale_pos_weight=1, # Balancing of positive and negative weights
               random_state=0,  # Random seed
               gpu_id=0,  # From nvidia-smi this is the gtx 1080ti
               max_bin=32, # For the GPU histogram method
               tree_method='gpu_hist' # XGBoost fast histogram (other option is gpu_exact)
            )

xgb = GridSearchCV(xgbmodel, param_grid)

start = datetime.now()

xgb.fit(X_train, y_train)

stop = datetime.now()

print("Ellapsed time = ", stop - start)
print('Best parameters found by grid search are:', xgb.best_params_)
print("Best score found by grid search is: ", xgb.best_score_)

Ellapsed time =  3:04:12.748602
Best parameters found by grid search are: {'learning_rate': 0.05, 'max_depth': 8, 'n_estimators': 85}
Best score found by grid search is:  0.432847338618


In [68]:
# Finally a grid search on Catboost
cb_param_grid = {'iterations': [50, 100, 300], 
                 'depth': [1, 2, 4],
                 'learning_rate': [0.01, 0.025, 0.05]
                }

cbmodel = CatBoostRegressor(loss_function='RMSE', device_config='GPU', task_type = 'GPU')

cb = GridSearchCV(cbmodel, cb_param_grid)

start = datetime.now()
cb.fit(X_train, y_train)
stop = datetime.now()

print("Ellapsed time = ", stop - start)
print('Best parameters found by grid search are:', cb.best_params_)
print("Best score found by grid search is: ", cb.best_score_)

0:	learn: 1.492206054	total: 107ms
	remaining: 5.23s1:	learn: 1.488980516	total: 127ms	remaining: 3.04s
2:	learn: 1.485789995	total: 146ms	remaining: 2.29s
3:	learn: 1.4826572	total: 166ms	remaining: 1.91s
4:	learn: 1.479579139	total: 186ms	remaining: 1.67s
5:	learn: 1.476540391	total: 205ms	remaining: 1.5s
6:	learn: 1.473544162	total: 224ms	remaining: 1.38s
7:	learn: 1.470601261	total: 244ms	remaining: 1.28s
8:	learn: 1.467704413	total: 266ms
	remaining: 1.21s9:	learn: 1.464847517	total: 292ms	remaining: 1.17s
10:	learn: 1.462040011	total: 312ms	remaining: 1.11s
11:	learn: 1.459266065	total: 333ms
	remaining: 1.05s12:	learn: 1.456540601	total: 353ms
	remaining: 1s13:	learn: 1.453835293	total: 372ms	remaining: 956ms
14:	learn: 1.451178738	total: 399ms	remaining: 931ms
15:	learn: 1.448545765	total: 421ms	remaining: 896ms
16:	learn: 1.445945808	total: 442ms	remaining: 858ms
17:	learn: 1.443388621	total: 461ms	remaining: 820ms
18:	learn: 1.44085693	total: 481ms	remaining: 784ms
19:	learn:

0:	learn: 1.487347764	total: 66.9ms
	remaining: 3.28s1:	learn: 1.479468364	total: 92ms	remaining: 2.21s
2:	learn: 1.471958113	total: 112ms	remaining: 1.75s
3:	learn: 1.464653857	total: 132ms
	remaining: 1.52s4:	learn: 1.457639615	total: 152ms
	remaining: 1.37s5:	learn: 1.450912489	total: 172ms
	remaining: 1.26s6:	learn: 1.444444014	total: 192ms	remaining: 1.18s
7:	learn: 1.438048192	total: 212ms
	remaining: 1.11s8:	learn: 1.43194138	total: 232ms
	remaining: 1.06s9:	learn: 1.425941378	total: 252ms	remaining: 1.01s
10:	learn: 1.420216037	total: 273ms	remaining: 968ms
11:	learn: 1.414607253	total: 293ms	remaining: 929ms
12:	learn: 1.409255604	total: 313ms	remaining: 891ms
13:	learn: 1.403955481	total: 334ms	remaining: 859ms
14:	learn: 1.398869356	total: 354ms
	remaining: 825ms15:	learn: 1.393870213	total: 374ms	remaining: 794ms
16:	learn: 1.38909954	total: 394ms	remaining: 764ms
17:	learn: 1.384400651	total: 413ms	remaining: 735ms
18:	learn: 1.379911152	total: 434ms	remaining: 708ms
19:	l

10:	learn: 1.366198616	total: 315ms	remaining: 1.12s
11:	learn: 1.358068005	total: 334ms	remaining: 1.06s
12:	learn: 1.350338349	total: 354ms	remaining: 1.01s
13:	learn: 1.343158184	total: 374ms	remaining: 963ms
14:	learn: 1.336372417	total: 393ms	remaining: 918ms
15:	learn: 1.329743564	total: 412ms	remaining: 875ms
16:	learn: 1.323576763	total: 430ms	remaining: 835ms
17:	learn: 1.317867276	total: 448ms	remaining: 797ms
18:	learn: 1.31219221	total: 466ms	remaining: 761ms
19:	learn: 1.307125554	total: 484ms	remaining: 726ms
20:	learn: 1.302041464	total: 502ms	remaining: 693ms
21:	learn: 1.297263189	total: 520ms	remaining: 661ms
22:	learn: 1.292806779	total: 538ms	remaining: 632ms
23:	learn: 1.288273933	total: 556ms	remaining: 602ms
24:	learn: 1.284169964	total: 578ms
	remaining: 578ms25:	learn: 1.280269166	total: 598ms
	remaining: 552ms26:	learn: 1.276526109	total: 617ms
	remaining: 526ms27:	learn: 1.272776426	total: 636ms	remaining: 500ms
28:	learn: 1.269241848	total: 656ms	remaining: 

	remaining: 1.73s20:	learn: 1.435928072	total: 452ms	remaining: 1.7s
21:	learn: 1.433477499	total: 470ms	remaining: 1.67s
22:	learn: 1.431081941	total: 489ms	remaining: 1.64s
23:	learn: 1.428695263	total: 508ms
	remaining: 1.61s24:	learn: 1.426363575	total: 527ms	remaining: 1.58s
25:	learn: 1.424037773	total: 547ms	remaining: 1.56s
26:	learn: 1.421755019	total: 568ms
	remaining: 1.53s27:	learn: 1.419497417	total: 587ms
	remaining: 1.51s28:	learn: 1.417287838	total: 607ms	remaining: 1.49s
29:	learn: 1.41508089	total: 625ms	remaining: 1.46s
30:	learn: 1.412906446	total: 644ms	remaining: 1.43s
31:	learn: 1.410769852	total: 662ms	remaining: 1.41s
32:	learn: 1.408643098	total: 680ms	remaining: 1.38s
33:	learn: 1.406555187	total: 698ms	remaining: 1.35s
34:	learn: 1.404484196	total: 716ms	remaining: 1.33s
35:	learn: 1.402423114	total: 733ms	remaining: 1.3s
36:	learn: 1.400392974	total: 752ms	remaining: 1.28s
37:	learn: 1.398402971	total: 769ms	remaining: 1.25s
38:	learn: 1.396415948	total: 78

80:	learn: 1.221671741	total: 1.62s	remaining: 380ms
81:	learn: 1.220424559	total: 1.64s	remaining: 360ms
82:	learn: 1.219186989	total: 1.66s	remaining: 340ms
83:	learn: 1.217976428	total: 1.68s	remaining: 320ms
84:	learn: 1.216774874	total: 1.7s	remaining: 300ms
85:	learn: 1.21556565	total: 1.72s	remaining: 280ms
86:	learn: 1.214376584	total: 1.74s	remaining: 260ms
87:	learn: 1.213202341	total: 1.76s	remaining: 240ms
88:	learn: 1.212052013	total: 1.78s
	remaining: 220ms89:	learn: 1.210909553	total: 1.8s	remaining: 200ms
90:	learn: 1.209781195	total: 1.82s	remaining: 180ms
91:	learn: 1.20863373	total: 1.84s	remaining: 160ms
92:	learn: 1.207499279	total: 1.86s
	remaining: 140ms93:	learn: 1.20640888	total: 1.88s
	remaining: 120ms94:	learn: 1.205299692	total: 1.9s	remaining: 100ms
95:	learn: 1.204207905	total: 1.92s	remaining: 80ms
96:	learn: 1.203134119	total: 1.94s	remaining: 60ms
97:	learn: 1.202061089	total: 1.96s	remaining: 39.9ms
98:	learn: 1.201004764	total: 1.98s	remaining: 20ms
9

...

235:	learn: 1.092991184	total: 8.31s
	remaining: 2.25s236:	learn: 1.092904116	total: 8.34s	remaining: 2.22s
237:	learn: 1.092793414	total: 8.38s
	remaining: 2.18s238:	learn: 1.092709159	total: 8.4s	remaining: 2.15s
239:	learn: 1.092599047	total: 8.45s	remaining: 2.11s
240:	learn: 1.092528869	total: 8.48s
	remaining: 2.08s241:	learn: 1.092360091	total: 8.51s	remaining: 2.04s
242:	learn: 1.092126244	total: 8.55s
	remaining: 2s243:	learn: 1.092087616	total: 8.58s	remaining: 1.97s
244:	learn: 1.092016961	total: 8.61s	remaining: 1.93s
245:	learn: 1.091878747	total: 8.64s	remaining: 1.9s
246:	learn: 1.091749953	total: 8.67s	remaining: 1.86s
247:	learn: 1.091417571	total: 8.71s
	remaining: 1.83s248:	learn: 1.091386194	total: 8.75s	remaining: 1.79s
249:	learn: 1.09132177	total: 8.79s	remaining: 1.76s
250:	learn: 1.09117208	total: 8.83s	remaining: 1.72s
251:	learn: 1.090259555	total: 8.86s
	remaining: 1.69s252:	learn: 1.090100934	total: 8.9s
	remaining: 1.65s253:	learn: 1.089963811	total: 8.93s

	remaining: 6.81s89:	learn: 1.013965731	total: 2.9s	remaining: 6.78s
90:	learn: 1.01371087	total: 2.94s
	remaining: 6.74s91:	learn: 1.013425264	total: 2.98s	remaining: 6.73s
92:	learn: 1.013203469	total: 3.01s
	remaining: 6.7s93:	learn: 1.012919993	total: 3.05s	remaining: 6.68s
94:	learn: 1.012739204	total: 3.08s	remaining: 6.64s
95:	learn: 1.012465836	total: 3.12s	remaining: 6.62s
96:	learn: 1.012298977	total: 3.14s	remaining: 6.58s
97:	learn: 1.012047529	total: 3.17s	remaining: 6.54s
98:	learn: 1.011819502	total: 3.21s
	remaining: 6.51s99:	learn: 1.011630548	total: 3.23s	remaining: 6.47s
100:	learn: 1.01134783	total: 3.27s	remaining: 6.45s
101:	learn: 1.011212615	total: 3.31s	remaining: 6.42s
102:	learn: 1.010989136	total: 3.35s	remaining: 6.4s
103:	learn: 1.010823112	total: 3.38s	remaining: 6.36s
104:	learn: 1.010677872	total: 3.4s	remaining: 6.32s
105:	learn: 1.010467651	total: 3.45s	remaining: 6.32s
106:	learn: 1.010130845	total: 3.49s	remaining: 6.29s
107:	learn: 1.009987966	tota

245:	learn: 0.9870355779	total: 8.33s	remaining: 1.83s
246:	learn: 0.9870021715	total: 8.36s	remaining: 1.79s
247:	learn: 0.9869188654	total: 8.39s	remaining: 1.76s
248:	learn: 0.9868711754	total: 8.42s	remaining: 1.73s
249:	learn: 0.9867884111	total: 8.46s	remaining: 1.69s
250:	learn: 0.9866912655	total: 8.49s	remaining: 1.66s
251:	learn: 0.9865543609	total: 8.53s	remaining: 1.62s
252:	learn: 0.9864875143	total: 8.56s	remaining: 1.59s
253:	learn: 0.9863893867	total: 8.6s	remaining: 1.56s
254:	learn: 0.9863051685	total: 8.63s
	remaining: 1.52s255:	learn: 0.9862519792	total: 8.66s
	remaining: 1.49s256:	learn: 0.9861848356	total: 8.69s	remaining: 1.45s
257:	learn: 0.9859893708	total: 8.73s
	remaining: 1.42s258:	learn: 0.9859081924	total: 8.76s	remaining: 1.39s
259:	learn: 0.9858322333	total: 8.79s	remaining: 1.35s
260:	learn: 0.9857105821	total: 8.83s	remaining: 1.32s
261:	learn: 0.9856813123	total: 8.86s	remaining: 1.28s
262:	learn: 0.9856188035	total: 8.89s	remaining: 1.25s
263:	learn:

	remaining: 6.68s99:	learn: 1.120515886	total: 3.35s	remaining: 6.7s
100:	learn: 1.120312642	total: 3.39s	remaining: 6.68s
101:	learn: 1.120043287	total: 3.43s
	remaining: 6.65s102:	learn: 1.119687259	total: 3.46s
	remaining: 6.62s103:	learn: 1.119380771	total: 3.5s	remaining: 6.59s
104:	learn: 1.119140656	total: 3.53s	remaining: 6.56s
105:	learn: 1.118983691	total: 3.57s	remaining: 6.53s
106:	learn: 1.118812403	total: 3.6s	remaining: 6.49s
107:	learn: 1.118565012	total: 3.64s	remaining: 6.46s
108:	learn: 1.118378268	total: 3.67s	remaining: 6.42s
109:	learn: 1.118146935	total: 3.69s	remaining: 6.38s
110:	learn: 1.117913006	total: 3.72s	remaining: 6.34s
111:	learn: 1.117636891	total: 3.75s	remaining: 6.3s
112:	learn: 1.117446086	total: 3.79s	remaining: 6.27s
113:	learn: 1.117064488	total: 3.83s	remaining: 6.24s
114:	learn: 1.116755436	total: 3.85s	remaining: 6.2s
115:	learn: 1.116556718	total: 3.88s	remaining: 6.16s
116:	learn: 1.116282059	total: 3.92s	remaining: 6.13s
117:	learn: 1.116

252:	learn: 1.090269096	total: 8.6s	remaining: 1.6s
253:	learn: 1.090199936	total: 8.63s	remaining: 1.56s
254:	learn: 1.090131605	total: 8.67s	remaining: 1.53s
255:	learn: 1.08998659	total: 8.7s	remaining: 1.5s
256:	learn: 1.089889883	total: 8.74s	remaining: 1.46s
257:	learn: 1.089844502	total: 8.77s	remaining: 1.43s
258:	learn: 1.089756572	total: 8.8s
	remaining: 1.39s259:	learn: 1.089658899	total: 8.84s	remaining: 1.36s
260:	learn: 1.089578908	total: 9.1s	remaining: 1.36s
261:	learn: 1.089473042	total: 9.14s
	remaining: 1.32s262:	learn: 1.089402663	
total: 9.17s	remaining: 1.29s263:	learn: 1.089329608	total: 9.2s	remaining: 1.25s
264:	learn: 1.08920029	total: 9.24s	remaining: 1.22s
265:	learn: 1.088725287	total: 9.28s	remaining: 1.19s
266:	learn: 1.08865898	total: 9.31s	remaining: 1.15s
267:	learn: 1.088535206	total: 9.34s
	remaining: 1.11s268:	learn: 1.088470279	total: 9.37s
	remaining: 1.08s269:	learn: 1.088306552	total: 9.41s	remaining: 1.04s
270:	learn: 1.088133498	total: 9.45s
	

Based upon the above grid searches, the best parameters are as follows:
1.  LightGBM - 'learning_rate': 0.05, 'n_estimators': 75
2.  XGBoost - 'learning_rate': 0.05, 'max_depth': 8, 'n_estimators': 85
3.  Catboost - 'depth': 1, 'iterations': 50, 'learning_rate': 0.01